## 1. Importing libraries

In [12]:
import numpy as np
import tensorflow as tf
import cv2
import os

## 2. Loading and Preprocessing Dataset

In [1]:
def load_hkr_data(data_dir):
    images, labels = [], []
    for label in sorted(os.listdir(data_dir)):
        label_dir = os.path.join(data_dir, label)
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 32)) / 255.0
            images.append(img)
            labels.append(label)
    return np.expand_dims(np.array(images), axis=-1), np.array(labels)

In [ ]:
data_path = ""
x_data, y_data = load_hkr_data(data_path)

## 3. Model fitting

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, LSTM, Reshape, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
inputs = Input(shape=(32, 128, 1))

In [5]:
x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)

In [6]:
x = MaxPooling2D(pool_size=(2,2))(x)

In [7]:
x = Reshape(target_shape=(-1, 32))(x)

In [8]:
x = Bidirectional(LSTM(128, return_sequences=True))(x)

In [9]:
x = Dense(100, activation='softmax')(x)

In [10]:
model = Model(inputs, x)
model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 128, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 128, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1024, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 1024, 256)      │       164,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024, 100)      │        25,700 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 190,884 (745.64 KB)

 Trainable params: 190,884 (745.64 KB)

 Non-trainable params: 0 (0.00 B)